In [ ]:
# Iran Black-Scholes Pricing on Brent Crude Options (BZ=F)

import pandas as pd
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
from datetime import datetime

# Load Brent options data
df = pd.read_csv("../data/brent_options_data.csv")
df = df.dropna(subset=["impliedVolatility"])

# Filter: out-of-the-money call options
df = df[(df["inTheMoney"] == False) & (df["strike"] > 75)]
df = df.head(50).copy()

# Assume Brent current price manually (or pull live via yfinance)
S = 83.0  # Example value
r = 0.05  # Annual risk-free rate

# Calculate time to expiration in years
def get_T(expiration_date):
    exp_date = datetime.strptime(expiration_date, "%Y-%m-%d")
    delta = (exp_date - datetime.today()).days
    return max(delta / 365, 1 / 365)

df["T"] = df["expirationDate"].apply(get_T)

# Black-Scholes model
def black_scholes(S, K, T, r, sigma, option_type="call"):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if option_type == "call":
        return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == "put":
        return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError("option_type must be 'call' or 'put'")

# Compute Black-Scholes prices
df["bs_price"] = df.apply(
    lambda row: black_scholes(S, row["strike"], row["T"], r, row["impliedVolatility"]),
    axis=1
)

# Calculate error vs. market price
df["abs_error"] = np.abs(df["bs_price"] - df["lastPrice"])

# Plot results
plt.figure(figsize=(10, 6))
plt.plot(df["strike"], df["bs_price"], label="Black-Scholes Price", color="darkred")
plt.scatter(df["strike"], df["lastPrice"], label="Market Price", color="black")
plt.xlabel("Strike Price")
plt.ylabel("Option Price")
plt.title("Brent Crude: Market vs. Black-Scholes Option Pricing (Iran Proxy)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Display table
print(df[["strike", "lastPrice", "bs_price", "abs_error"]].head(10))
